In [14]:
import sys
sys.path.append('../../..') 
from openai import OpenAI
from tqdm import tqdm
import json
import os
import numpy as np
from MultimodalForcast.data_loader.data_loader import process_bitcoin_data, split_series, TimeSeriesDataset, collate_fn

In [18]:
def gpt4_forecast(prediction_window, system_prompt, user_prompt):
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=64,
        temperature=0.0,
    )
    forecast_text = response.choices[0].message.content.strip()
    try:
        forecast = [float(x) for x in forecast_text.split(",")]
        # Ensure the forecast is the correct length
        if len(forecast) != prediction_window:
            forecast = forecast[:prediction_window] + [float('nan')] * (prediction_window - len(forecast))
            print("forcast length does not match prediction window")
    except Exception:
        forecast = [float('nan')] * prediction_window
        print("forcast is in invalid format")
    return forecast

In [19]:

# data parameters
start_date = '2018-01-01'
end_date = '2022-04-22'
ts_file_path = '../../data/bitcoin/bitcoin_daily.csv'
news_file_path = '../../data/bitcoin/bitcoin_news_with_summaries.json'
text_col = 'summary'
lookback = 15
predict = 7
batch_size= 16

df_filtered = process_bitcoin_data(ts_file_path, news_file_path, start_date, end_date, text_col=text_col)
train_data, val_data, test_data = split_series(df_filtered, lookback)
val_dataset = TimeSeriesDataset(val_data, lookback, predict, text_col=text_col)
val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=False, collate_fn=collate_fn)

There are 1236 rows in the filtered bitcoin dataframe


In [32]:
# Initialize lists to store predictions and targets
all_preds = []
all_targets = []

# Define prompts
system_prompt = (
    "You are an expert financial forecaster specializing in Bitcoin. "
    "Given recent Bitcoin-related news (as a JSON list, each with a 'summary_i' field) and the last N days of Bitcoin prices, "
    "your job is to predict the next M day(s) of Bitcoin price. "
    "Reply ONLY with a comma-separated list of the forecasted Bitcoin prices."
)

# Loop through validation loader
for batch in tqdm(val_loader):
    texts = batch['text']  # list of lists of news summaries (per lookback day)
    values = batch['value']  # tensor: (batch, lookback, features)
    targets = batch['target']  # tensor: (batch, predict, features)
    
    # Process each sample in the batch
    for i in range(len(texts)):
        # Build news JSON list for this sample
        news_json_list = []
        for idx, summary in enumerate(texts[i]):
            news_json_list.append({
                f"summary_{idx+1}": summary
            })
        news_json = json.dumps(news_json_list, ensure_ascii=False)
        lookback_values = values[i, :, 0].tolist()  # assuming 1D value
        
        # Create user prompt for this sample
        user_prompt = (
            f"News (JSON): {news_json}\n"
            f"Recent Bitcoin prices: {lookback_values}\n"
            f"Forecast the next {predict} day(s) of Bitcoin price as a comma-separated list."
        )
        
        # Get forecast from GPT-4 using your existing function
        forecast = gpt4_forecast(predict, system_prompt, user_prompt)
        
        # Store predictions and targets
        all_preds.append(forecast)
        all_targets.append(targets[i, :, 0].tolist())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

# Compute RMSE
rmse = np.sqrt(np.nanmean((all_preds - all_targets) ** 2))
print(f"Validation RMSE: {rmse:.4f}")

# Optional: Print per-step RMSE
for step in range(predict):
    step_rmse = np.sqrt(np.nanmean((all_preds[:, step] - all_targets[:, step]) ** 2))
    print(f"Step {step+1} RMSE: {step_rmse:.4f}")

100%|██████████| 12/12 [06:14<00:00, 31.19s/it]

Validation RMSE: 513.5371
Step 1 RMSE: 221.1346
Step 2 RMSE: 338.7480
Step 3 RMSE: 426.7504
Step 4 RMSE: 500.8677
Step 5 RMSE: 572.5100
Step 6 RMSE: 641.5729
Step 7 RMSE: 714.1596
